## Introduction

In [ ]:
'''
Deep Learning workflows

1. Data
2. Create a Model
3. Optimize Model paramter (finding the best weights)
4. Save the trained model
'''

## Importing the libraries

In [1]:
import torch #pytorch library that helps in building the deep leanring algorithms
from torch import nn ##nn means neural network
from torch.utils.data import DataLoader #performs the process of batching
from torchvision import datasets #Inbuild dataset that pytorch library has - FashionMnist Dataset
from torchvision.transforms import ToTensor

## Download the data

In [2]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 12474223.07it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 212612.83it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3895773.25it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5706204.28it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

## Batching of this data

In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [12]:
for X,y in test_dataloader: #Image - Color image shape (batch_size,number of channel,length,width)
  print(X.shape)            #Image - Black and white image - number of channels is 1
  print(y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


## Creating the model

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu" #torch.cuda.is_available() checks for your system has gpu or cpu
device

'cuda'

In [14]:
class NeuralNetwork(nn.Module): #Child class and nn.Module is a parent class -- (defined in the pytorch library)

  def __init__(self): #declare the architecture
    super().__init__() #basically initailizes all the variables of the parent class
    self.flatten = nn.Flatten() #28x28 image into a 764x1 vector
    self.linear1 = nn.Linear(28*28,512)
    self.linear2 = nn.Linear(512,512)
    self.linear3 = nn.Linear(512,10)
    self.relu = nn.ReLU()
  def forward(self,x):#is always used to pass the inputs to the neural network
    x = self.flatten(x)
    x = self.linear1(x)
    x = self.relu(x)
    x = self.linear2(x)
    x = self.relu(x)
    x = self.linear3(x)
    return x

In [15]:
model = NeuralNetwork()
model = model.to(device) #copies your entire architecture to the GPU

## Optimization - Gradient Descent + backpropogation

In [25]:
loss_fn = nn.CrossEntropyLoss() #cross entropy loss
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3) #stochastic Gradient descent

In [26]:
#steps in the GD : Batch of the input / Pass it to the model / Compute loss function / Update the weights

def train(dataloader,model,loss_fn,optimizer):
  model.train() #putting the model in the training mode
  for batch,(X,y) in enumerate(dataloader):
    #sending the data to the GPU
    X = X.to(device)
    y = y.to(device)

    #Compute predictions
    pred = model(X)

    #Compute loss
    loss = loss_fn(pred,y)

    #Backpropogation
    loss.backward() #Wnew = Wold - lr*dl/dw
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print(f'Loss of the Model {loss.item()}')

In [29]:
def test(dataloader,model,loss_fn):
  model.eval() #putting the model in the training mode
  num_batched = len(dataloader)
  test_loss, correct = 0,0
  with torch.no_grad(): #We will not compute gradients for the test data
    for X,y in dataloader:
      X = X.to(device)
      y = y.to(device)

      #Compute predictions
      pred = model(X)

      #Compute loss
      test_loss += loss_fn(pred,y).item()

      #Find how many correct predictions
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss = test_loss/num_batched
  correct = correct/(len(dataloader.dataset))

  print(f'Test Accuracy {100*correct}, Avg_loss : {test_loss}')

## Training Phase

In [30]:
epochs = 5

for t in range(5):
  print(f'Epoch {t+1}')
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)

Epoch 1
Loss of the Model 1.4052774906158447
Loss of the Model 1.3908345699310303
Loss of the Model 1.2259663343429565
Loss of the Model 1.3262871503829956
Loss of the Model 1.2031605243682861
Loss of the Model 1.2123459577560425
Loss of the Model 1.2250869274139404
Loss of the Model 1.1572368144989014
Loss of the Model 1.2026647329330444
Loss of the Model 1.1122525930404663
Test Accuracy 63.89, Avg_loss : 1.1402978570597946
Epoch 2
Loss of the Model 1.2081586122512817
Loss of the Model 1.2136220932006836
Loss of the Model 1.0320074558258057
Loss of the Model 1.1627304553985596
Loss of the Model 1.0343506336212158
Loss of the Model 1.0588557720184326
Loss of the Model 1.0868233442306519
Loss of the Model 1.0234606266021729
Loss of the Model 1.0730667114257812
Loss of the Model 0.9979758858680725
Test Accuracy 65.35, Avg_loss : 1.0176877838790797
Epoch 3
Loss of the Model 1.0765368938446045
Loss of the Model 1.103747010231018
Loss of the Model 0.902263343334198
Loss of the Model 1.05799

## Save the Model

In [31]:
torch.save(model.state_dict(),"/content/Weights/iteration1.pth")

## Loading the saveed weights

In [32]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/content/Weights/iteration1.pth"))

<All keys matched successfully>

## Predictions

In [37]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle Boot"]

model.eval()
X,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  X = X.to(device)
  pred = model(X)
  predicted,actual = classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted {predicted}')
  print(f'Actual {actual}')

Predicted Ankle Boot
Actual Ankle Boot


In [35]:
y

9